<a href="https://colab.research.google.com/github/s-grzhang/gwc-chatbot/blob/main/gwc_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch

In [ ]:
import pandas as pd
from datasets import Dataset
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the DataFrame from the CSV file
file_path = '/content/drive/My Drive/gwc_chatbot_data_revised.csv'
df = pd.read_csv(file_path)

# Combine User Input and Bot Response into a single text format suitable for training
df['text'] = df['User Input'] + " " + df['Bot Response']

# Create a Dataset from the DataFrame
dataset = Dataset.from_pandas(df[['text']])

# Display the dataset
print(dataset)

Mounted at /content/drive
Dataset({
    features: ['text'],
    num_rows: 98
})


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add a padding token to the tokenizer (GPT-2 doesn't have a native pad token)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset into train and test sets
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.043900,2.650371
2,2.212100,2.451590
3,1.608300,2.380935
4,1.317500,2.346972
5,1.022600,2.370211
6,0.809500,2.447189
7,0.664300,2.421088
8,0.541200,2.444559
9,0.442200,2.480185
10,0.527500,2.483770


TrainOutput(global_step=390, training_loss=1.2335238444499481, metrics={'train_runtime': 2343.2996, 'train_samples_per_second': 0.333, 'train_steps_per_second': 0.166, 'total_flos': 50951946240000.0, 'train_loss': 1.2335238444499481, 'epoch': 10.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/My Drive/gwc_transfer_learning_chatbot_model')
tokenizer.save_pretrained('/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer')

('/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer/vocab.json',
 '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer/merges.txt',
 '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer/added_tokens.json',
 '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

# Load the tokenizer and model
tokenizer_name = '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer'
model_name = '/content/drive/My Drive/gwc_transfer_learning_chatbot_model'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Test the chatbot
def get_response(user_input):
    inputs = tokenizer.encode(user_input, return_tensors="pt")
    outputs = model.generate(inputs, max_length=63, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

user_input = "When does the first meeting take place?"
response = get_response(user_input)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


When does the first meeting take place? The first meeting takes place after the Club Jamboree, which usually takes place in September. The second meeting takes place in October. Each club meeting lasts about 30 minutes. Each club member is given a brief presentation about the club and its initiatives, along with a brief
